# Teste pyomo

In [80]:
import numpy as np
from udgp import Instance

In [81]:
instance = Instance.artificial_molecule(5, freq=True, seed=123456)
instance.view_input()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [82]:
import pyomo.environ as pyo

In [83]:
model = pyo.ConcreteModel()

instance.reset()

model.n = pyo.Param(initialize=instance.n)
model.m = pyo.Param(initialize=instance.m)

## CONJUNTOS
ny = 1
rng = np.random.default_rng()
y_indices = rng.choice(instance.fixed_n, ny, replace=False)

nx = 4
x_indices = np.arange(instance.fixed_n, nx + instance.fixed_n)

model.Iy = pyo.Set(initialize=y_indices)
model.Ix = pyo.Set(initialize=x_indices)
model.I = model.Iy | model.Ix

k = np.arange(instance.m)[instance.freqs != 0]
model.K = pyo.Set(initialize=k)

model.IJxx = pyo.Set(
    within=model.Ix * model.Ix,
    initialize=((i, j) for i in model.Ix for j in model.Ix if i < j),
)
model.IJyx = pyo.Set(
    initialize=model.Iy * model.Ix
)
model.IJ = model.IJyx | model.IJxx

print(model.IJ.data())


((0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4))


In [67]:
model.L = pyo.Set(initialize=[0, 1, 2])

## PARÂMETROS
model.max_gap = pyo.Param(initialize=0.01)

model.d_max = pyo.Param(initialize=instance.dists.max())
model.d_min = pyo.Param(initialize=instance.dists.min())

model.dists = pyo.Param(
    model.K, 
    within=pyo.PositiveReals, 
    initialize=instance.dists
)
model.freqs = pyo.Param(
    model.K, 
    within=pyo.NonNegativeIntegers, 
    initialize=instance.freqs,
)

model.y = pyo.Param(
    model.Iy, 
    model.L, 
    within=pyo.Reals,
    initialize={(i, l): instance.points[i, l] for l in model.L for i in model.Iy}
)

## VARIÁVEIS BASE
model.a = pyo.Var(
    model.IJ, 
    model.K, 
    within=pyo.Binary,
)
model.x = pyo.Var(
    model.Ix, 
    model.L, 
    within=pyo.Reals
)
model.v = pyo.Var(
    model.IJ, 
    model.L, 
    within=pyo.Reals
)
model.r = pyo.Var(
    model.IJ, 
    within=pyo.NonNegativeReals,
    bounds=(model.d_min, model.d_max),
)

## VARIÁVEIS MACULAN
model.p = pyo.Var(
    model.IJ, 
    model.K, 
    within=pyo.Reals,
    bounds=(-model.max_gap, model.max_gap),
)
model.w = pyo.Var(
    model.IJ, 
    model.K, 
    within=pyo.NonNegativeReals,
    bounds=(0, model.max_gap),
)
model.z = pyo.Var(
    model.IJ, 
    model.K, 
    within=pyo.NonNegativeReals,
    bounds=(0, model.d_max),
)



In [73]:
## RESTRIÇÕES BASE
@model.Constraint(model.K)
def constr_a1(model, k):
    return sum(model.a[i, j, k] for i, j in model.IJ) <= model.freqs[k]


@model.Constraint(model.IJ)
def constr_a2(model, i, j):
    return sum(model.a[i, j, k] for k in model.K) == 1


@model.Constraint(model.IJxx, model.L)
def constr_vxx(model, i, j, l):
    return model.v[i, j, l] == model.x[j, l] - model.x[i, l]


@model.Constraint(model.IJyx, model.L)
def constr_vxy(model, i, j, l):
    return model.v[i, j, l] == model.y[i, l] - model.x[j, l]


@model.Constraint(model.IJ)
def constr_r(model, i, j):
    return model.r[i, j] ** 2 == sum(model.v[i, j, l] ** 2 for l in model.L)


## RESTRIÇÕES MACULAN
@model.Constraint(model.IJ, model.K)
def constr_x1(model, i, j, k):
    return model.w[i, j, k] >= model.p[i, j, k]


@model.Constraint(model.IJ, model.K)
def constr_x2(model, i, j, k):
    return model.w[i, j, k] >= -model.p[i, j, k]


@model.Constraint(model.IJ, model.K)
def constr_x3(model, i, j, k):
    return model.z[i, j, k] >= model.r[i, j] - model.d_max * (1 - model.a[i, j, k])


@model.Constraint(model.IJ, model.K)
def constr_x4(model, i, j, k):
    return model.z[i, j, k] <= model.r[i, j] + model.d_max * (1 - model.a[i, j, k])


@model.Constraint(model.IJ, model.K)
def constr_x5(model, i, j, k):
    return model.z[i, j, k] >= -model.d_max * model.a[i, j, k]


@model.Constraint(model.IJ, model.K)
def constr_x6(model, i, j, k):
    return model.z[i, j, k] <= model.d_max * model.a[i, j, k]


@model.Constraint(model.IJ, model.K)
def constr_x7(model, i, j, k):
    return model.z[i, j, k] >= model.dists[k] + model.p[i, j, k] - model.d_max * (1 - model.a[i, j, k])


@model.Constraint(model.IJ, model.K)
def constr_x8(model, i, j, k):
    return model.z[i, j, k] <= model.dists[k] + model.p[i, j, k] + model.d_max * (1 - model.a[i, j, k])

# @model.Constraint(model.IJ, model.K)
# def constr_x9(model, i, j, k):
#     return model.w[i, j, k] <= model.dists[k] * 0.05


## OBJETIVO
@model.Objective(sense=pyo.minimize)
def objective(model):
    return 1 + pyo.summation(model.w)


In [74]:
print(model.max_gap * len(list(model.IJ.data())))

0.1


In [79]:
instance.reset()

opt = pyo.SolverFactory('gurobi_direct')

opt.options['NonConvex'] = 2
opt.options['MIPGap'] = 0.1
opt.options['IntFeasTol'] = 1e-4
opt.options['FeasibilityTol'] = 1e-4
opt.options['OptimalityTol'] = 1e-4

opt.solve(model, tee=True, report_timing=True)


        0.05 seconds required for presolve
Set parameter NonConvex to value 2
Set parameter MIPGap to value 0.1
Set parameter IntFeasTol to value 0.0001
Set parameter FeasibilityTol to value 0.0001
Set parameter OptimalityTol to value 0.0001
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 445 rows, 252 columns and 1178 nonzeros
Model fingerprint: 0x256f6729
Model has 10 quadratic constraints
Variable types: 202 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-02, 4e+00]
  RHS range        [5e-01, 8e+00]
Presolve removed 82 rows and 22 columns
Presolve time: 0.01s
Presolved: 493 rows, 270 columns, 1564 nonzeros
Presolved model has 40 bilinear constraint(s)
Vari

{'Problem': [{'Name': 'unknown', 'Lower bound': 1.0, 'Upper bound': 1.009275146505341, 'Number of objectives': 1, 'Number of constraints': 455, 'Number of variables': 252, 'Number of binary variables': 50, 'Number of integer variables': 50, 'Number of continuous variables': 152, 'Number of nonzeros': 1178, 'Sense': 1, 'Number of solutions': 2}], 'Solver': [{'Name': 'Gurobi 10.02', 'Status': 'ok', 'Wallclock time': 2.4283699989318848, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [76]:
points = np.array([[model.x[i, l].value for l in model.L] for i in model.Ix])
print(points)

[[ 0.22104264  2.02085224  1.3660297 ]
 [-1.42247456  3.8236759   1.14852052]
 [ 0.56447485  2.80972309  0.13733234]
 [ 0.02986435  3.50858413  1.35216596]]


In [77]:
instance.add_points(points)

True

In [78]:
print(instance.is_solved())
instance.view()

True


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol